<a href="https://colab.research.google.com/github/andys0tc/ML2018/blob/master/Autocorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementar un sistema de autocorrector.


Funciones a realizar:


*   Conteo de palabras en un corpus
*   Obtener probabilidad de palabra en el corpus
*   Maniputar cadenas
*   Filtrar cadenas
*   Implementar distancia mínima para comparar cadena y encontrar el camino más corto
*   Progración dimámica





## Edit Distance

Implementar modelo que corrija palabras a 1 o 2 de distancia.

*    Dos palabras estan a n distancia de edición cuando necesitamos  $n$ movimineto de una palabra a otra


*   Eliminar una letra => 'hat' => 'at','ha','ht,...
*   Voltear 2 letras adjuntas=> 'eta' => 'eat', 'tea',...
*   Reemplazar una letra =>'jat' =>'hat','rat','cat',...
*   Insertar una letra =>'te' =>'tea','ten','ate',...


# Preparar Datos


In [ ]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Para la función procesar datos:


1.   Leer corpus
2.   Convertir a minúsculas
3.   Regresar lista de palabras



In [ ]:
def process_data(file_name):
 
    words = [] # return this variable correctly
        
    file = open(file_name, 'r')
    Lines = file.readlines()

    for line in Lines:
        line = line.lower()
        line = re.sub('[^a-zA-Z0-9]'," ",line)
        text=line.split()
        for word in text:
            words.append(word)
    
    return words

In [14]:
word_l = process_data('/content/drive/MyDrive/semester work/shakespeare.txt')

vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


Regresa un diccionario donde el key es la palabra y el value el número de veces que aparece en el corpus.

In [ ]:
def get_count(word_l):
      
    word_count_dict = {}  
    for word in word_l:
        if word in word_count_dict:
            word_count_dict[word]+=1
        else:
            word_count_dict[word] = 1
        
    return word_count_dict

In [16]:
word_count_dict = get_count(word_l)
print(word_count_dict)

{'o': 157, 'for': 474, 'a': 757, 'muse': 18, 'of': 1094, 'fire': 22, 'that': 785, 'would': 138, 'ascend': 1, 'the': 1525, 'brightest': 2, 'heaven': 48, 'invention': 9, 'kingdom': 6, 'stage': 3, 'princes': 4, 'to': 1196, 'act': 35, 'and': 1395, 'monarchs': 2, 'behold': 13, 'swelling': 1, 'scene': 34, 'then': 132, 'should': 102, 'warlike': 4, 'harry': 1, 'like': 101, 'himself': 23, 'assume': 1, 'port': 3, 'mars': 12, 'at': 131, 'his': 434, 'heels': 3, 'leash': 1, 'd': 321, 'in': 787, 'hounds': 2, 'famine': 3, 'sword': 17, 'crouch': 1, 'employment': 2, 'but': 420, 'pardon': 20, 'gentles': 1, 'all': 258, 'flat': 1, 'unraised': 1, 'spirits': 7, 'have': 329, 'dared': 1, 'on': 178, 'this': 355, 'unworthy': 5, 'scaffold': 1, 'bring': 32, 'forth': 25, 'so': 352, 'great': 89, 'an': 95, 'object': 3, 'can': 103, 'cockpit': 1, 'hold': 28, 'vasty': 1, 'fields': 3, 'france': 50, 'or': 196, 'may': 108, 'we': 189, 'cram': 1, 'within': 23, 'wooden': 1, 'very': 49, 'casques': 1, 'did': 110, 'affright': 1

#Conteo de probabilidades

Dado el diccionario con las frecuencia calcular la probabilidad de aparición  en el corpus
$
P(w_i)=C(W_i)/M$

In [ ]:
def get_probs(word_count_dict):
 
    probs = {}  # return this variable correctly
    
    M = sum(word_count_dict.values())
    for word in word_count_dict:
        probs[word]=word_count_dict[word]/M

    return probs

In [17]:
sum(word_count_dict.values())

53614

In [19]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


#Manipulación de palabras


*   Eliminar una letra 
*   Voltear 2 letras adjuntas
*   Reemplazar una letra
*   Insertar una letra

In [ ]:
def delete_letter(word, verbose=False):
     
    delete_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    delete_l = [(A+B[1:]) for A,B in split_l if B]

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [20]:
delete_word_l = delete_letter(word="cans",
                        verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's'), ('cans', '')], 
delete_l = ['ans', 'cns', 'cas', 'can']


In [ ]:
def switch_letter(word, verbose=False):
     
    switch_l = []
    split_l = []
        
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    switch_l = [( A[:]+B[1]+B[0]+B[2:]) for A, B in split_l if len(B)>1]
        
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [21]:
switch_word_l = switch_letter(word="eta",
                         verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a'), ('eta', '')] 
switch_l = ['tea', 'eat']


In [ ]:
def replace_letter(word, verbose=False):

    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    replace_set=set()
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    for A,B in split_l:
        if B:
            for l in letters:
                replace_set.add(A[:]+l+B[1:])
    replace_set.remove(word)
      
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [22]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n'), ('can', '')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [ ]:
def insert_letter(word, verbose=False):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
     split_l = [(word[:i], word[i:]) for i in range(len(word)+1 )]
    for A, B in split_l:
        for l in letters:
            insert_l.append(A[:] + l + B[:])
    
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [23]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")


Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


# Combinar ediciones

La función regresará todos los posibles ediciones sobre una cadena

In [ ]:
def edit_one_letter(word, allow_switches = True):
    
    edit_one_set = set()
    
    complete = []
    complete.extend(delete_letter(word)) #delete
    complete.extend(replace_letter(word)) #replace
    complete.extend(insert_letter(word)) #insert 
    if allow_switches:
        complete.extend(switch_letter(word))
        
    return set(complete)

In [33]:
def edit_two_letters(word, allow_switches = True):
    
    edit_two_set = set()
    
    second_edition = sorted(edit_one_letter(word,allow_switches))

    try:
        second_edition.remove("")
    except:
        pass
    for new_word in second_edition:
        all_strings = edit_one_letter(new_word,allow_switches)
        edit_two_set|= all_strings
    
    return edit_two_set

In [26]:
proof_word = "et"
proof_edit_one_set = edit_one_letter(proof_word)
# turn this into a list to sort it, in order to view it
proof_edit_one_letter = sorted(list(proof_edit_one_set))

print(f"input word {proof_word} \nedit_one_l \n{proof_edit_one_letter}\n")
print(f"The type of the returned object should be a set {type(proof_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word et 
edit_one_l 
['aet', 'at', 'bet', 'bt', 'cet', 'ct', 'det', 'dt', 'e', 'ea', 'eat', 'eb', 'ebt', 'ec', 'ect', 'ed', 'edt', 'ee', 'eet', 'ef', 'eft', 'eg', 'egt', 'eh', 'eht', 'ei', 'eit', 'ej', 'ejt', 'ek', 'ekt', 'el', 'elt', 'em', 'emt', 'en', 'ent', 'eo', 'eot', 'ep', 'ept', 'eq', 'eqt', 'er', 'ert', 'es', 'est', 'eta', 'etb', 'etc', 'etd', 'ete', 'etf', 'etg', 'eth', 'eti', 'etj', 'etk', 'etl', 'etm', 'etn', 'eto', 'etp', 'etq', 'etr', 'ets', 'ett', 'etu', 'etv', 'etw', 'etx', 'ety', 'etz', 'eu', 'eut', 'ev', 'evt', 'ew', 'ewt', 'ex', 'ext', 'ey', 'eyt', 'ez', 'ezt', 'fet', 'ft', 'get', 'gt', 'het', 'ht', 'iet', 'it', 'jet', 'jt', 'ket', 'kt', 'let', 'lt', 'met', 'mt', 'net', 'nt', 'oet', 'ot', 'pet', 'pt', 'qet', 'qt', 'ret', 'rt', 'set', 'st', 't', 'te', 'tet', 'tt', 'uet', 'ut', 'vet', 'vt', 'wet', 'wt', 'xet', 'xt', 'yet', 'yt', 'zet', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


#Sugerencia de ortografía

Usamos la función edit_two_letters para obtener un conjunto de todas las posibles palabras a de 2 ediciones

In [27]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    
    suggestions = []
    n_best = []
    
    c = Counter()

    if(word in vocab): #word is in vocabulary
        c[word]=probs[word]
        
    else:
        one_edit = edit_one_letter(word)  
        if(len(one_edit)):
            for word_one in one_edit:
                if (word_one in vocab):
                    c[word_one] = probs[word_one]
        if(len(c)<n):
            two_edit = edit_two_letters(word) 
            if len(two_edit):
                for word_two in two_edit:
                    if (word_two in vocab):
                        c[word_two] = probs[word_two]

                 
        if(not c):
            c[word] = 1
        
    n_best = c.most_common(n)
    suggestions = [word for word, word_count in n_best]
    
    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [ ]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
 
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1, m+1):  # Replace None with the proper range
        D[row, 0] = D[row-1, 0]+1

    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1, n+1):  # Replace None with the proper range
        D[0, col] = D[0, col-1]+1

    # Loop through row 1 to row m, both inclusive
    for row in range(1, m+1):

        # Loop through column 1 to column n, both inclusive
        for col in range(1, n+1):

            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost

            # Check to see if source character at the previous row
            # matches the target character at the previous column,
            l_source = list(source)
            l_target = list(target)
            if l_source[row-1]==l_target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0

            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row, col] = min(D[row-1,col] + del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost)

    # Set the minimum edit distance with the cost found at row m, column n

    med = D[m,n]
    
    return D, med

In [35]:
my_word = 'toughts' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")

entered word =  toughts 
suggestions =  ['thought', 'thoughts']
word 0: thought, probability 0.000653
word 1: thoughts, probability 0.000616
data type of corrections <class 'list'>
